# Computational Theory Assessment

In [6]:
import numpy as np

## <Strong>Problem 1 </strong>: Binary Words and Operations

> **AI Assistance Disclosure**  
> The test code block structures (for `Parity`, `Choose`, `Maj`, and all Σ/σ functions) were **AI-generated using ChatGPT (2025)** to maintain consistent formatting and binary visualisation style across functions.  
> All logic and final verification were completed manually.


### 1. Parity 
**Parity** is defined in the [Secure Hash Standard (FIPS 180-4, § 4.1.1)](https://nvlpubs.nist.gov/nistpubs/FIPS/NIST.FIPS.180-4.pdf) as:

$$
\text{Parity}(x, y, z) = x \oplus y \oplus z
$$

where ⊕ shows the **bitwise XOR** operation.

For each bit position, the result is 1 if an **odd number** of x, y and z contain a 1 in that position, otherwise 0.  
Parity acts as an **odd-bit detector**, ensuring that even a single bit change in the inputs alters the output.

**Simply Said:** Do I have an odd number of 1s(INPUT 1) or an even number of 1s(INPUT 0)?

**Why XOR is used**
- **XOR** stands for *exclusive OR*. 
- It's a logical operation that compares two bits and outputs **1 if they are different** and **0 if they are the same**.
- XOR is **fast**, **branch-free**, and supported directly in Hardware.  
- It implements addition mod 2 at the bit level.  

**Alternative forms in research**

Some implementations describe *Parity* differently:

| Formulation | Description | Why I Didn't Use|
| :-- | :-- | :-- |
| `(x + y + z) % 2` | Modulo-2 sum - Simplistic show of odd/even nature by summing bits and taking the remainder mod 2. |  You’d need to apply to each bit - expensive for 32-bit words. |
| `bin(x ^ y ^ z).count("1") % 2`  | Bit Count Method - parity of all bits within one number| Very slow, string conversion, not bitwise.|


**Truth Table (for 1-bit inputs)**

| x | y | z | Parity |
|:--:|:--:|:--:|:--:|
| 0 | 0 | 0 | 0 |
| 0 | 0 | 1 | 1 |
| 0 | 1 | 0 | 1 |
| 0 | 1 | 1 | 0 |
| 1 | 0 | 0 | 1 |
| 1 | 0 | 1 | 0 |
| 1 | 1 | 0 | 0 |
| 1 | 1 | 1 | 1 |

In [7]:
# Uses np.uint32() : https://numpy.org/devdocs/user/basics.types.html
def parity(x: np.uint32, y: np.uint32, z: np.uint32):
    """
    Return the parity (XOR) of three 32-bit numbers.

    Each bit of the result is 1 if an odd number of the inputs
    have a 1 in that position, otherwise 0.

    Parameters:
        x (uint32): First 32-bit integer.
        y (uint32): Second 32-bit integer.
        z (uint32): Third 32-bit integer.

    Returns:
        np.uint32: The XOR (parity) of the three input values.
    """
    return np.uint32(x ^ y ^ z)


In [8]:
# Initiate Test values
x = np.uint32(0b10101010)
y = np.uint32(0b11001100)
z = np.uint32(0b01111000)

# Show original inputs
print("              x =", np.binary_repr(x, 32))
print("              y =", np.binary_repr(y, 32))
print("              z =", np.binary_repr(z, 32))
 
# Perform Parity
result = parity(x, y, z)

# Show result in binary
# Uses np.binary_repr(): https://numpy.org/doc/2.1/reference/generated/numpy.binary_repr.html
print("Parity(x, y, z) =", np.binary_repr(result, 32))

              x = 00000000000000000000000010101010
              y = 00000000000000000000000011001100
              z = 00000000000000000000000001111000
Parity(x, y, z) = 00000000000000000000000000011110


### 2. Choose(x, y, z)

**Choose** is defined in the [Secure Hash Standard (FIPS 180-4, § 4.1.2)](https://nvlpubs.nist.gov/nistpubs/FIPS/NIST.FIPS.180-4.pdf) as:

$$
\text{Ch}(x, y, z) = (x \land y) \oplus (\lnot x \land z)
$$

Where:  
- `∧` is the **bitwise AND** operation,  
- `⊕` is the **bitwise XOR**,  
- `¬` is the **bitwise NOT** (flips every bit).  

**Objective**  
Choose uses `x` as a **bit-selector** - for every bit position:  
- if bit from x = 1, take the corresponding bit from y;  
- if bit from x = 0, take it from z.  

So x acts like a 32-bit “key” that chooses between y and z.


**Simply said:** **Choose** picks bits from `y` or `z` depending on whether `x`’s bit is 1 or 0.


**Alternative forms and interpretations**

| Formulation | Description | Why I Didn't Use|
|:--|:--|:--|
| `(x & y) \| (~x & z)` | OR instead of XOR | Non-Uniformity within problem set  |
| `z ^ (x & (y ^ z))` | Algebraic | Doesn't Match Spec, Not Great Readability |

**Why this method (`np.uint32((x & y) ^ (~x & z))`)?**  
- It exactly follows the standard equation.  
- **np.uint32** confines results to 32 bits (no overflow).  
- Bitwise operations are hardware-level fast and branch-free — each bit is processed independently.  


**Example (1-bit truth table)**  

| x | y | z | Choose(x,y,z) |
|:-:|:-:|:-:|:-------------:|
| 0 | 0 | 0 | 0 |
| 0 | 0 | 1 | 1 |
| 0 | 1 | 0 | 0 |
| 0 | 1 | 1 | 1 |
| 1 | 0 | 0 | 0 |
| 1 | 0 | 1 | 0 |
| 1 | 1 | 0 | 1 |
| 1 | 1 | 1 | 1 |


In [9]:
def choose(x: np.uint32, y: np.uint32, z: np.uint32):
    """
    Return the result of choosing bits from y and z based on x.
    
    For each bit position, if the bit in x is 1, take the bit from y,
    otherwise take the bit from z.

    bitwise NOT operator (~) flips each bit: 0 becomes 1, and 1 becomes 0.
    bitwise AND operator (&) compares each bit of two numbers and returns 1 if both bits are 1, otherwise returns 0.
    bitwise XOR operator (^) compares each bit of two numbers and returns 1 if the bits are different, otherwise returns 0.

    Parameters:
        x (uint32): The mask.
        y (uint32): The first number.
        z (uint32): The second number.
    """
    return np.uint32((x & y) ^ (~x & z))


In [10]:
# Initiate Test values
x = np.uint32(0b10101010)
y = np.uint32(0b11001100)
z = np.uint32(0b01111000)

# Show original inputs
print("             x  =", np.binary_repr(x, 32))
print("             y  =", np.binary_repr(y, 32))
print("             z  =", np.binary_repr(z, 32))

# Perform Choose
result = choose(x, y, z)

# Show result in binary
print("Choose(x, y, z) =", np.binary_repr(result, 32))


             x  = 00000000000000000000000010101010
             y  = 00000000000000000000000011001100
             z  = 00000000000000000000000001111000
Choose(x, y, z) = 00000000000000000000000011011000


### 3. Maj(x, y, z)

Defined in the [Secure Hash Standard (FIPS 180-4, § 4.1.2)](https://nvlpubs.nist.gov/nistpubs/FIPS/NIST.FIPS.180-4.pdf) as:

$$
\text{Maj}(x, y, z) = (x \land y) \oplus (x \land z) \oplus (y \land z)
$$


**Objective**  
The **Majority** function outputs 1 for each bit position where **two or more**
of the corresponding bits in `x`, `y`, and `z` are 1.
This makes it act as a “bitwise decider,” combining information from multiple inputs
to strengthen the *diffusion* of bits.


**Simply said:**  Majority - whichever value (0 or 1) occurs most often across x, y, and z.

**Alternative forms and interpretations**

| Formulation | Description | Why I Didn't Use|
|:--|:--|:--|
| `(x & y) \| (x & z) \| (y & z)` | Equivalent with OR instead of XOR | Less conventional - XOR keeps consistency with other SHA logical functions. |
| `(x & y) \| (z & (x \| y))` | Boolean simplification | Harder to read - hides the “majority” meaning behind extra grouping. |
| `((x ^ y) & z) \| (x & y)` | Optimized algebraic rearrangement |  Used for performance in C, unclear for teaching purposes & no scalability |


**Why this method (`np.uint32((x & y) ^ (x & z) ^ (y & z))`)?**  
- It directly mirrors the **official FIPS definition**, ensuring correctness and traceability.  
- Using `np.uint32` confines operations to 32-bit words, matching SHA-256’s internal word size.  
- The XOR version maintains consistency with the rest of SHA’s logic functions, which are all expressed with XOR, AND, and NOT.  
- It’s branch-free, efficient, and clear.

**Example (1-bit inputs)**  

| x | y | z | Maj(x,y,z) |
|:-:|:-:|:-:|:-----------:|
| 0 | 0 | 0 | 0 |
| 0 | 0 | 1 | 0 |
| 0 | 1 | 1 | 1 |
| 1 | 0 | 1 | 1 |
| 1 | 1 | 0 | 1 |
| 1 | 1 | 1 | 1 |


In [11]:
def Maj(x: np.uint32, y: np.uint32, z: np.uint32):
    """
    Compute the majority function.

    Maj(x, y, z) = (x & y) ^ (x & z) ^ (y & z)

    For each bit position, the output is 1 if two or more of
    the corresponding bits in x, y, and z are 1, otherwise 0.

    Parameters
    ----------
    x, y, z : np.uint32
        32-bit unsigned integers.

    Returns
    -------
    np.uint32
        The bitwise majority of x, y, and z.
    """
    return (x & y) ^ (x & z) ^ (y & z)

In [12]:
# Initiate Test values
x = np.uint32(0b10101010)
y = np.uint32(0b11001100)
z = np.uint32(0b01111000)

# Show original inputs
print("           x =", np.binary_repr(x, 32))
print("           y =", np.binary_repr(y, 32))
print("           z =", np.binary_repr(z, 32))

# Perform Majority
result = Maj(x, y, z)

# Show result in binary
print("Maj(x, y, z) =", np.binary_repr(result, 32))


           x = 00000000000000000000000010101010
           y = 00000000000000000000000011001100
           z = 00000000000000000000000001111000
Maj(x, y, z) = 00000000000000000000000011101000


#### Helper Functions 
- I chose to use **helper functions** (`ROTR` and `SHR`) for a clean, encapsulated, and       efficient implementation.  This keeps these operations reusable across all of the Σ and σ functions, ensures 32-bit consistency,  and makes the code easier to test and understand.

In [13]:
# Helper functions for Sigma functions
# Rotate-right operation for 32-bit words - Helper function for Sigma functions
def ROTR(x: np.uint32, n: int) -> np.uint32:
    """Rotate-right operation for 32-bit words."""
    return np.uint32((x >> n) | (x << np.uint32(32 - n)))

# Shift-right operation for 32-bit words - Helper function for Sigma functions
def SHR(x: np.uint32, n: int) -> np.uint32:
    """Shift-right operation for 32-bit words."""
    return np.uint32(x >> n)


### 4. Sigma Zero - Σ₀(x) 

Defined in the [Secure Hash Standard (FIPS 180-4, §4.1.2)](https://nvlpubs.nist.gov/nistpubs/FIPS/NIST.FIPS.180-4.pdf) as:

$$
\Sigma_0^{\{256\}}(x) = \text{ROTR}^2(x) \oplus \text{ROTR}^{13}(x) \oplus \text{ROTR}^{22}(x)
$$

**Objective**  
The **Σ₀** function performs three right-rotations of the 32-bit `x`
by 2, 13, and 22 bits, then XORs the results.

This process ensures that bits from different positions in `x`
influence each other in later stages of the SHA-256 computation,
creating strong *diffusion* — a key cryptographic property
where small changes in input cause large, unpredictable changes in output.


**Simply Said:**  Σ₀(x) “scrambles” the bits of x by spinning them around and combining the three versions with XOR to mix them thoroughly.


**Alternative forms and interpretations**

| Formulation | Description | Why I Didn’t Use |
|:--|:--|:--|
| `np.right_shift` with masking | Implements shifts manually | More verbose and less readable than `ROTR()` helper. |

**Why I Chose This Method**  
- The XOR-based rotation exactly matches the FIPS specification.  
- Using the shared `ROTR()` helper keeps all rotations consistent and clear across Σ₀, Σ₁, σ₀, and σ₁.  
- NumPy’s `np.uint32` enforces 32-bit wrapping automatically, avoiding overflow or sign issues.  
- It’s fast & readable.


**Behaviour Summary**
| Operation | Description |
|------------|-------------|
| ROTR²(x)  | Circular right rotation by 2 bits |
| ROTR¹³(x) | Circular right rotation by 13 bits |
| ROTR²²(x) | Circular right rotation by 22 bits |

The final result is the XOR (⊕) of these three rotated words.


In [14]:
def Sigma0(x: np.uint32) -> np.uint32:
    """
    Compute the Σ₀ (Sigma zero) function used in SHA-256.

    Defined in FIPS 180-4 (Section 4.1.2) as:
        Σ₀(x) = ROTR²(x) ⊕ ROTR¹³(x) ⊕ ROTR²²(x)

    Parameters
    x : np.uint32
        32-bit unsigned integer input word.

    Returns
    np.uint32
        The resulting 32-bit word after applying Σ₀(x).
    """
    return ROTR(x, 2) ^ ROTR(x, 13) ^ ROTR(x, 22)


In [15]:
# Initiate Test value
x = np.uint32(0b10110010)


# Display result in binary for readability
# Uses np.binary_repr(): https://numpy.org/doc/2.1/reference/generated/numpy.binary_repr.html
print("Original Number:", np.binary_repr(x, 32))
print("Sigma0 Result  :", np.binary_repr(Sigma0(x), 32))


Original Number: 00000000000000000000000010110010
Sigma0 Result  : 10000101100100101100100000101100


### 5. Sigma One - Σ₁(x) 

Defined in the [Secure Hash Standard (FIPS 180-4, §4.1.2)](https://nvlpubs.nist.gov/nistpubs/FIPS/NIST.FIPS.180-4.pdf) as:

$$
\Sigma_1^{\{256\}}(x) = \text{ROTR}^6(x) \oplus \text{ROTR}^{11}(x) \oplus \text{ROTR}^{25}(x)
$$


**Objective**  
The **Σ₁** function performs three right rotations of the 32-bit word `x`
by 6, 11, and 25 bits, then XORs the results.
This operation spreads information across bits and rounds, ensuring
that small differences in the input word produce large, unpredictable
differences in the output. It enhances *diffusion* and *non-linearity*,
both essential for cryptographic strength in SHA-256.

**Simply Said:** Σ₁(x) twists the bits of `x` three times by different amounts, then combines them.


**Alternative forms and interpretations**

| Formulation | Description | Why I Didn’t Use |
|:--|:--|:--|
| `(x >> 6) ^ (x >> 11) ^ (x >> 25)` | Simple shifts instead of rotations | Loses information — bits are discarded instead of wrapped around. |
| `(ROTR(x,6) + ROTR(x,11) + ROTR(x,25)) & 0xFFFFFFFF` | Integer addition, not XOR | Incorrect — adds values numerically instead of mixing bits logically. |

---

**Why I Chose This Method**  
- The XOR-based rotation exactly matches the FIPS specification.  
- Using the shared `ROTR()` helper keeps all rotations consistent and clear across Σ₀, Σ₁, σ₀, and σ₁.  
- NumPy’s `np.uint32` enforces 32-bit wrapping automatically, avoiding overflow or sign issues.  
- It’s fast & readable.

**Behaviour Summary**
| Operation | Description |
|------------|--------------|
| ROTR⁶(x)  | Circular right rotation by 6 bits |
| ROTR¹¹(x) | Circular right rotation by 11 bits |
| ROTR²⁵(x) | Circular right rotation by 25 bits |

The final output is the XOR (⊕) of these three rotated words.


In [16]:
def Sigma1(x: np.uint32) -> np.uint32:
    """
    Compute the Σ₁ (Sigma one) function.

    This function performs three right-rotations of a 32-bit word `x` by
    6, 11, and 25 bits, and returns the bitwise XOR of these results:

        Σ₁(x) = ROTR⁶(x) ⊕ ROTR¹¹(x) ⊕ ROTR²⁵(x)

    Parameters
    x : np.uint32
        32-bit unsigned integer input word.

    Returns
    np.uint32
        The resulting 32-bit word after applying Σ₁(x).
    """
    return ROTR(x, 6) ^ ROTR(x, 11) ^ ROTR(x, 25)


In [17]:
# Initiate Test value
x = np.uint32(0b10110010)

# Show original input
print("x     =", np.binary_repr(x, 32))

# Perform Σ₁(x)
result = Sigma1(x)



# Display the result in binary for readability
# Uses np.binary_repr(): https://numpy.org/doc/2.1/reference/generated/numpy.binary_repr.html
print("Σ₁(x) =", np.binary_repr(Sigma1(x), 32))


x     = 00000000000000000000000010110010
Σ₁(x) = 11011110010000000101100100000010


### 6. Small Sigma Zero - σ₀(x) 

Defined in the [Secure Hash Standard (FIPS 180-4, § 4.1.2)](https://nvlpubs.nist.gov/nistpubs/FIPS/NIST.FIPS.180-4.pdf) as:

$$
\sigma_0^{\{256\}}(x) = \text{ROTR}^7(x) \oplus \text{ROTR}^{18}(x) \oplus \text{SHR}^3(x)
$$


**Objective**  
σ₀(x) is one of the *message schedule* functions used in SHA-256.  
It takes a 32-bit x, rotates it right by 7 and 18 bits, then performs a simple right shift by 3 bits (filling with zeros).  

The three results are XORed together to create a new, intensely mixed version of `x`.  
This helps improving **diffusion** (the spreading of something more widely).

**Simply Said:**  σ₀(x) spins 'x' around twice and gives it one shove to the right. The result is then XOR'd : XOR (exclusive OR) outputs 1 when bits differ, and 0 when they’re the same.


**Alternative forms and interpretations**

| Formulation | Description | Why I Didn’t Use |
|:--|:--|:--|
| `(x >> 7) ^ (x >> 18) ^ (x >> 3)` | All logical shifts | Loses information - shifts discard bits instead of rotating them. |
| `np.roll()` on bit arrays | Simulates rotation on arrays | Slower  |


**Why I Chose This Method**  
- Follows the exact equation defined in FIPS 180-4 for SHA-256.   
- I chose to use **helper functions** (`ROTR` and `SHR`) for a clean, encapsulated, and       efficient implementation.  This keeps these operations reusable across all of the Σ and σ functions, ensures 32-bit consistency,  and makes the code easier to test and understand.
- NumPy’s `np.uint32` keeps all arithmetic confined to 32 bits, matching the real hardware behaviour.  
- It’s simple & readable.


**Example (8-bit demo)**  
(Shown shorter for readability — real SHA-256 uses 32 bits.)

| Step | Operation | Result |
|:--|:--|:--|
| Input | x = `10010011` |  |
| 1 | ROTR⁷(x) | `11001001` |
| 2 | ROTR¹⁸(x) | `01100100` |
| 3 | SHR³(x) | `00010010` |
| XOR all | σ₀(x) | `10111111` |

The final output is the XOR (⊕) of these three results.


In [18]:
def sigma0(x: np.uint32) -> np.uint32:
    """
    Compute the σ₀ (sigma zero) function used in SHA-256.

    Defined in FIPS 180-4 (Section 4.1.2) as:
        σ₀(x) = ROTR⁷(x) ⊕ ROTR¹⁸(x) ⊕ SHR³(x)

    This function performs two right-rotations followed by one right-shift,
    then XORS the results together.

    Parameters
    x : np.uint32
        32-bit unsigned integer input word.
   
    Returns
    np.uint32
        The resulting 32-bit word after applying σ₀(x).
    """

    return np.bitwise_xor(np.bitwise_xor(ROTR(x, 7), ROTR(x, 18)), SHR(x, 3))

In [19]:
#Testing
# Initiate Test value
x = np.uint32(0b10110010)

# Show original input
print("x     =", np.binary_repr(x, 32))

# Perform σ₀(x)
result = sigma0(x)

# Show result in binary
print("σ₀(x) =", np.binary_repr(result, 32))


x     = 00000000000000000000000010110010
σ₀(x) = 01100100001011001000000000010111


### σ₁(x) — sigma One

Defined in the [Secure Hash Standard (FIPS 180-4, § 4.1.2)](https://nvlpubs.nist.gov/nistpubs/FIPS/NIST.FIPS.180-4.pdf) as:

$$
\sigma_1^{\{256\}}(x) = \operatorname{ROTR}^{17}(x)\ \oplus\ \operatorname{ROTR}^{19}(x)\ \oplus\ \operatorname{SHR}^{10}(x)
$$

**Objective**  
Performs two **bitwise right rotations** and one **logical right shift** on a 32-bit word by 17, 19, and 10 bits respectively, then XORs the results together.  
This function contributes to **diffusion** in the **SHA-256 message schedule**, expanding message words so that every bit influences later rounds.

**Simply Said:**  
σ₁(x) spins `x` twice, slides it once to the right, and then combines them using **XOR** — where XOR (exclusive OR) outputs 1 only when the bits differ.

**Alternative forms and interpretations**  
- Sometimes written as  
  $$
  \sigma_1(x) = (x \ggg 17)\ \oplus\ (x \ggg 19)\ \oplus\ (x \gg 10)
  $$  
  where $(\ggg)$ means *rotate right* (ROTR) and $(\gg)$ means *logical shift right* (SHR).  
- The difference between rotate and shift: **rotate** wraps bits around, while **shift** discards bits that move past the edge and fills with zeros.

**Why I chose this method**  
I used **helper functions** (`ROTR` and `SHR`) for a clean, encapsulated, and efficient implementation.  
This keeps these operations reusable across all of the Σ and σ functions, ensures 32-bit consistency, and makes the code easier to test and understand.

**Example (8-bit demo)**  
(Shown shorter for readability — real SHA-256 uses 32 bits.)

| Step | Operation | Result |
|:--|:--|:--|
| Input | x = 10110010 |  |
| 1 | ROTR¹⁷(x) | 01011001 |
| 2 | ROTR¹⁹(x) | 10010110 |
| 3 | SHR¹⁰(x) | 00001011 |
| XOR all | σ₁(x) | 11010100 |

The final output is the XOR (⊕) of these three results, demonstrating how σ₁(x) mixes the rotated and shifted versions of `x` to achieve diffusion.


In [20]:
def sigma1(x: np.uint32) -> np.uint32:
    """
    σ₁(x) — Lowercase sigma one 

    Computes: ROTRⁱ⁷(x) XOR ROTR¹⁹(x) XOR SHR¹⁰(x)

    Parameters
    x : np.uint32
        32-bit input word.

    Returns
    np.uint32
        32-bit result of σ₁(x).

    """
    return ROTR(x, 17) ^ ROTR(x, 19) ^ SHR(x, 10)


In [21]:
# Initiate Test value
x = np.uint32(0b10110010)

# Show original input
print("x     =", np.binary_repr(x, 32))

# Perform σ₁(x)
result = sigma1(x)

# Show result in binary
print("σ₁(x) =", np.binary_repr(result, 32))


x     = 00000000000000000000000010110010
σ₁(x) = 00000000010011110100000000000000


## <strong>Problem 2 </strong>: Fractional Parts of Cube Roots

### <strong>Prime Number</strong> Function

#### Definition
A **prime number** is a positive integer greater than 1 that has no positive factors other than 1 and itself
([Khan Academy, 2022](https://www.khanacademy.org/math/arithmetic/factors-multiples/prime-numbers/v/recognizing-prime-numbers)).


For example, `2`, `3`, `5`, and `7` are prime numbers, while  `8` is not because it has multiple divisors (2 & 4).

The `primes(n)` function I created generates the **first _n_ prime numbers**, starting from 2.  
It uses **trial division**, where each candidate number is tested for divisibility by every integer up to its square root.  
This approach is conceptually simple and more than efficient enough for small values like the first 64 primes used in SHA-256.

---

#### Other Implementations
There are several alternative ways to check or generate prime numbers in Python:

| Formulation | Description | Why I Didn’t Use |
| :-- | :-- | :-- |
| **Sieve of Eratosthenes** | Efficient algorithm that marks multiples of primes in a range, producing all primes up to a limit. | Adds complexity when only 64 primes are needed.  ([GeeksforGeeks, 2022](https://www.geeksforgeeks.org/python/python-program-for-sieve-of-eratosthenes/)). |
| **Built-in Libraries (`sympy.primerange`)** | Uses external packages that can instantly list primes. | External libraries aren’t permitted for this assessment to my knowledge. ([SymPy Documentation](https://docs.sympy.org/latest/modules/ntheory.html)). |
| **Optimized Trial Division** | Same logic as mine but skips even numbers above 2 and divides only by previously found primes. | Faster but less readable. |

---

#### Why I Chose This Method
I chose the **basic trial division** method because:
- It uses only **NumPy**.  
- It’s **simple and readable**, making it easy to explain and test.  
- It quickly handles up to **n = 64**, which is all we need for the SHA-256 constants.  
- It clearly demonstrates the logic behind prime checking.

It’s not the fastest, but it’s the clearest!

---

#### Run Through of Method
1. Start with an empty list `primes_list` and set the first candidate number to 2.  
2. For each candidate, assume it’s prime (`is_prime = True`).  
3. Loop through all numbers from `2` to the **square root** of the candidate (`np.sqrt(candidate)`):  
   - If any of those divides evenly, mark `is_prime = False`.  
4. If still prime, append the candidate to the list.  
5. Increase the candidate by 1 and repeat until we have `n` primes.  
6. Return the list of primes.


In [22]:
# Step One: Generate first n prime numbers
def primes(n):
    """
    Return the first n prime numbers using simple trial division.

    Parameters:
        n (int): The number of prime numbers to generate.
    """
    primes_list = []
    candidate = 2

    while len(primes_list) < n:
        is_prime = True
        for i in range(2, int(np.sqrt(candidate)) + 1):
            if candidate % i == 0:
                is_prime = False
                break
        if is_prime:
            primes_list.append(candidate)
        candidate += 1

    return primes_list

# Example : Test to see first 64 primes are correct
print(primes(64))  

[2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47, 53, 59, 61, 67, 71, 73, 79, 83, 89, 97, 101, 103, 107, 109, 113, 127, 131, 137, 139, 149, 151, 157, 163, 167, 173, 179, 181, 191, 193, 197, 199, 211, 223, 227, 229, 233, 239, 241, 251, 257, 263, 269, 271, 277, 281, 283, 293, 307, 311]


### <strong>Cube Root</strong> Calculation

#### Definition
**Cube roots** are the inverse operation of cubing a number, meaning they help us find which number, when multiplied by itself three times, gives the original number ([GeeksforGeeks, 2023](https://www.geeksforgeeks.org/maths/how-to-teach-cube-roots/)). 

For example, the cube root of 8 is 2 because $$ 2^3 = 8 $$  
In this problem, the cube roots of the first 64 prime numbers are used to generate constants for the SHA-256 algorithm - constants in SHA-256 are fixed numbers built into the algorithm.
They are used in the inner mathematical loops of the hash to mix bits together!

Each cube root gives a number like **1.259921...**, and it’s the **fractional part** (the numbers after the decimal) that we’ll use later to form the constants.

---

#### Other Implementations

| Method | Description | Why I Didn’t Use |
| :-- | :-- | :-- |
| **Newton–Raphson Method** | An iterative algorithm that refines an estimate of the cube root using the formula $$ x_{k+1} = \frac{2x_k + n/x_k^2}{3} $$ | Accurate but more complex & NumPy already provides a precise function. |

 - Reference: [Flexiple – Newton–Raphson Method in Python](https://flexiple.com/python/newton-raphson-method-python)
---

#### Why I Chose This Method
I used **NumPy’s `np.cbrt()`** function because it:
- Is **numerically stable**.  
- Handles both positive and negative inputs correctly.  
- Accurate results.  
- Clean and readable code for this small dataset.
([NumPy Documentation – `numpy.cbrt`](https://numpy.org/doc/stable/reference/generated/numpy.cbrt.html)).
---

#### Run Through of Method
1. Generate the first 64 prime numbers using the `primes(n)` function.  
2. Pass the list of primes into NumPy’s cube root function:
   ```python
   cube_roots = np.cbrt(prime_list)
   ```
   
3. Print first 10.


In [23]:
# Step Two: Compute cube roots of first 64 primes
prime_list = primes(64)
cube_roots = np.cbrt(prime_list)

# Display first few results
for i, val in enumerate(cube_roots[:10]):
    print(f"Prime {prime_list[i]:>3}  : Cube root = {val:.15f}")


Prime   2  : Cube root = 1.259921049894873
Prime   3  : Cube root = 1.442249570307408
Prime   5  : Cube root = 1.709975946676697
Prime   7  : Cube root = 1.912931182772389
Prime  11  : Cube root = 2.223980090569316
Prime  13  : Cube root = 2.351334687720758
Prime  17  : Cube root = 2.571281590658235
Prime  19  : Cube root = 2.668401648721945
Prime  23  : Cube root = 2.843866979851565
Prime  29  : Cube root = 3.072316825685847


### <strong>Fractional Parts of Cube Roots</strong>

#### Definition
The **fractional part** of a number is the part after the decimal point.  
For example, the fractional part of `3.1415` is ``0.1415``.  
The fractional parts of the cube roots of the first 64 primes are extracted, these will then be used to generate the 32-bit constants for SHA-256.

Equation:
$$
\text{fractional part} = x - \lfloor x \rfloor
$$

([MathWorld – Fractional Part Function](https://mathworld.wolfram.com/FractionalPart.html))

---

#### Why This Step Is Needed
The fractional parts ensure that the constants are **non-repeating & uniformly distributed**,  
providing randomness without relying on arbitrary values. 

According to the **Secure Hash Standard**  
([NIST FIPS 180-4, Section 4.2.2](https://nvlpubs.nist.gov/nistpubs/FIPS/NIST.FIPS.180-4.pdf)),  
these constants are derived from the fractional parts of the cube roots of prime numbers.  

This guarantees that the constants are *predictable yet unbiased*

---

#### Run Through of Method
1. Compute cube roots of the first 64 primes.  
2. Extract the fractional part of each cube root:
   ```python
   fractional_parts = cube_roots % 1
   ```


In [24]:
# Step Three: Extract fractional parts of cube roots
fractional_parts = cube_roots % 1 

# Display first few fractional parts
for i in range(10):
    print(f"Prime {prime_list[i]:>3} : Fractional Part = {fractional_parts[i]:.15f}")


Prime   2 : Fractional Part = 0.259921049894873
Prime   3 : Fractional Part = 0.442249570307408
Prime   5 : Fractional Part = 0.709975946676697
Prime   7 : Fractional Part = 0.912931182772389
Prime  11 : Fractional Part = 0.223980090569316
Prime  13 : Fractional Part = 0.351334687720758
Prime  17 : Fractional Part = 0.571281590658235
Prime  19 : Fractional Part = 0.668401648721945
Prime  23 : Fractional Part = 0.843866979851565
Prime  29 : Fractional Part = 0.072316825685847


### <strong>Creating 32-bit Constants</strong>

#### Overview
This final step converts the fractional parts of the cube roots into **32-bit integer constants**.  
Each fractional value is multiplied by $$ 2^{32} $$ to extract the first 32 binary digits.  
NumPy’s `np.floor()` function truncates (not rounds) the result, keeping only the integer portion.  
The array is then cast to `np.uint32` to match the 32-bit word size used in the **SHA-256 algorithm**.

#### Function: `creatingConstants()`
This function:
1. Multiplies each fractional part by $$ 2^{32} $$  
2. Uses `np.floor()` to drop the fractional component.  
3. Converts the values into unsigned 32-bit integers.  
4. Prints the first ten constants in both decimal and hexadecimal formats for verification.  
5. Returns all 64 constants as a NumPy array.

These constants form the **K₀ – K₆₃ table** defined in the Secure Hash Standard and  
are used during each of the 64 rounds of SHA-256.

#### References
- National Institute of Standards and Technology (2015).  
  *Secure Hash Standard (SHS)*, FIPS PUB 180-4.  
  [https://nvlpubs.nist.gov/nistpubs/FIPS/NIST.FIPS.180-4.pdf](https://nvlpubs.nist.gov/nistpubs/FIPS/NIST.FIPS.180-4.pdf)  
- NumPy Documentation. *numpy.floor — Return the floor of the input*  
  [https://numpy.org/doc/stable/reference/generated/numpy.floor.html](https://numpy.org/doc/stable/reference/generated/numpy.floor.html)



In [25]:
def creatingConstants(fractional_parts, primes):
    """
    Step Four: Convert fractional parts to 32-bit integer constants.

    Each fractional part is multiplied by 2**32 to extract its first 32 binary
    digits. The `np.floor()` function drops everything after the decimal point,
    so only the integer portion remains. The final array is cast to `np.uint32`
    to match the 32-bit word size used in SHA-256.

    Parameters
    ----------
    fractional_parts : numpy.ndarray
        The fractional parts of the cube roots of the first 64 prime numbers.
    primes : list
        The list of prime numbers - for demonstrating.

    Returns
    -------
    numpy.ndarray
        Array of unsigned 32-bit integers representing the SHA-256 constants.
    """
    constants = np.floor(fractional_parts * (2**32)).astype(np.uint32)

    # Display first few results for verification
    print("Prime   Constant (decimal)   Constant (hex)")
    print("--------------------------------------------")
    for i in range(10):
        dec_val = constants[i]
        hex_val = f"{dec_val:08x}"
        print(f"{primes[i]:>5}   {dec_val:>15}         {hex_val}")

    return constants


constants = creatingConstants(fractional_parts, prime_list)



Prime   Constant (decimal)   Constant (hex)
--------------------------------------------
    2        1116352408         428a2f98
    3        1899447441         71374491
    5        3049323471         b5c0fbcf
    7        3921009573         e9b5dba5
   11         961987163         3956c25b
   13        1508970993         59f111f1
   17        2453635748         923f82a4
   19        2870763221         ab1c5ed5
   23        3624381080         d807aa98
   29         310598401         12835b01


### <strong>Comparison with Official SHA-256 Constants</strong>

To confirm the correctness of the generated constants, the first eight values are compared  
against the official SHA-256 constants listed in the Secure Hash Standard (FIPS 180-4, page 11).  

Each comparison checks if the hexadecimal constant from the last few cells 
matches the corresponding published value from NIST.

A result of `: True` means the generated constant is identical to the official one,  
confirms the implementation is correct.


In [26]:
# Compare with official SHA-256 constants from FIPS 180-4
official_hex = [
    "428a2f98", "71374491", "b5c0fbcf", "e9b5dba5",
    "3956c25b", "59f111f1", "923f82a4", "ab1c5ed5"
]

for i in range(8):
    mine = f"{constants[i]:08x}"
    print(f"{i}: {mine} == {official_hex[i]} : {mine == official_hex[i]}")


0: 428a2f98 == 428a2f98 : True
1: 71374491 == 71374491 : True
2: b5c0fbcf == b5c0fbcf : True
3: e9b5dba5 == e9b5dba5 : True
4: 3956c25b == 3956c25b : True
5: 59f111f1 == 59f111f1 : True
6: 923f82a4 == 923f82a4 : True
7: ab1c5ed5 == ab1c5ed5 : True


## **Problem 3:** Padding

### Padding 

Before a message can be processed by the SHA-256 algorithm, it must be **padded** so that its total length is a multiple of **512 bits (64 bytes)**.  This is to prepare the data before pushing it to the hashing algorithm.

This ensures the algorithm can handle *any* message size in fixed-sized chunks.
 
**Padding** process as defined in the [Secure Hash Standard (FIPS 180-4, § 5.1.1, 5.2.1)](https://nvlpubs.nist.gov/nistpubs/FIPS/NIST.FIPS.180-4.pdf) works as follows:  

1. Start with the message of length (L) bits.  
2. Add a single `1` bit (`0x80` in hex, that marks the end of the original data.  
3. Add enough `0` bits so that the total length ≡ 448 (mod 512).  
   – This leaves 64 bits (8 bytes) at the end to note the message length.  
4. Add the **64-bit big-endian** representation of L (the original bit length).  
   – “Big-endian” means the most significant byte is written first. 
5. The result is one or more **512-bit blocks**, ready for hashing.

These steps guarantee every padded message is exactly 512-bits. 


## Simply Said
Padding makes sure every message fits perfectly into 512-bit blocks so SHA-256 can process it correctly.
It labels itself with the original message, adds a single **“1”** to mark its done, fills the space with **buffer zeros**, and finishes by writing down its **length** at the end!

## Reference
- [*How Does SHA-256 Work?* – Learn Me A Bitcoin (YouTube)](https://www.youtube.com/watch?v=f9EbD6iY9zI)
   Clear visual explanation of padding, including the `1` bit, zero padding, and length field.
- [*Python Generators: A Complete Guide* – Real Python](https://realpython.com/introduction-to-python-generators/)  
  Used to explain how `yield` works and why the function returns one block at a time.
- [*Endianness Explained (Big vs Little Endian)* – GeeksForGeeks](https://www.geeksforgeeks.org/little-and-big-endian-mystery/)  
  Helps explain what “64-bit big-endian length field” means in SHA-256 padding.
- [*ChatGPT (OpenAI)*](https://chatgpt.com/) – Interactive explanations, guidance, markdown used throughout this solution.  


## Example of Message being padded
Lets take the binary number : 
| Step | Description | Result (in bits / hex) |
|:----:|:-------------|:-----------------------|
| 1 | Message | `01100011 01100001 01110100` (`cat`) |
| 2 | Add ‘1’ bit | `...01110100 1` → adds `0x80` |
| 3 | Pad with zeros to reach 448 bits | `00…00` until total = 448 bits |
| 4 | Append 64-bit length (24 in binary) | `00000000 00000000 ... 00011000` |
| 5 | Final padded message = 512 bits (64 bytes) | Ready to split into a single block |



In [27]:
def block_parse(msg: bytes):
    """
    Generator function that pads and splits a message into 512-bit (64-byte) blocks
    according to the SHA-256 rules described in FIPS 180-4.

    Parameters
    ----------
    msg : bytes
        The input message as a bytes object.

    Returns
    ------
    padded : bytes
        The manipulated message after initial padding.
    """
    # Get original message length in bits
    msg_length_bits = len(msg) * 8

    # Add the single '1' bit (0x80 in hex) 
    padded = msg + b'\x80' #10000000

    # Calculate how many zero bytes are needed
    # We need the total length (in bits) ≡ 448 mod 512
    # 64 bytes (the total) minus 8 bytes (the length) = 56 bytes available for data and padding
    while (len(padded) % 64) != 56:
        padded += b'\x00'

    # Add the 64-bit big-endian length
    padded += msg_length_bits.to_bytes(8, 'big')

    # return the padded message
    return padded





In [28]:
#Testing

msg = b"cat"  # 3 bytes = 24 bits
padded = block_parse(msg)

print("Original message (bytes):", msg)
print("Padded message length (bytes):", len(padded))
print("Padded message length (bits):", len(padded) * 8)

# Print hex representation (trimmed)
print("\nPadded message (hex):")
print(padded.hex())

# Print last 8 bytes - the message length in bits
print("\nLast 8 bytes (hex):", padded[-8:].hex())


Original message (bytes): b'cat'
Padded message length (bytes): 64
Padded message length (bits): 512

Padded message (hex):
63617480000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000018

Last 8 bytes (hex): 0000000000000018


### Splitting into 512-bit Blocks  

**Split the padded message into 512-bit (64-byte) blocks**.  

Each block will later be processed separately during the hashing stage (Problem 4),  
so this function will be a **generator** that *yields one block at a time* instead of returning the entire padded message.  

- **Yield**: pauses and gives you one piece of data each time you ask for it.

Simply said — we’re about to turn our padded message into small, neat chunks that SHA-256 can process one at a time.


In [ ]:
def block_parse(msg: bytes):
    """
    Generator function that pads and splits a message into 512-bit (64-byte) blocks
    according to the SHA-256 rules described in FIPS 180-4.
    """
    # Step 1: Original message length in bits
    msg_length_bits = len(msg) * 8

    # Step 2: Append the single '1' bit (0x80)
    padded = msg + b'\x80'

    # Step 3: Pad with zeros until total length ≡ 56 mod 64 (bytes)
    while (len(padded) % 64) != 56:
        padded += b'\x00'

    # Step 4: Append the 64-bit big-endian length
    padded += msg_length_bits.to_bytes(8, 'big')

    # Step 5: Yield each 512-bit (64-byte) block
    for i in range(0, len(padded), 64): # starts at 0, goes to length of padded, step by 64
        yield padded[i:i + 64] # yields one block at a time


In [30]:
# Testing the generator version of block_parse
test_msg = b"catsarethebestpetstohaveinthewholewideworldandtheyareverycuteandfluffy"  

print("Original message:", test_msg)
print("Original length (bits):", len(test_msg) * 8)
print("=" * 60)

# adds a counter - block1, block2, ...
for index, block in enumerate(block_parse(test_msg), start=1):
    print(f"Block {index}:")
    print("  Length (bytes):", len(block))
    print("  Length (bits):", len(block) * 8)
    print("  Hex preview:", block.hex())
    print("-" * 60)


Original message: b'catsarethebestpetstohaveinthewholewideworldandtheyareverycuteandfluffy'
Original length (bits): 560
Block 1:
  Length (bytes): 64
  Length (bits): 512
  Hex preview: 636174736172657468656265737470657473746f68617665696e74686577686f6c6577696465776f726c64616e64746865796172657665727963757465616e64
------------------------------------------------------------
Block 2:
  Length (bytes): 64
  Length (bits): 512
  Hex preview: 666c7566667980000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000230
------------------------------------------------------------


## <strong>Problem 4 </strong>: Hashes

## Hashes

In this problem we implement the **SHA-256 compression function**, based on   
[Section 6.2.2 of the Secure Hash Standard (FIPS 180-4)](https://nvlpubs.nist.gov/nistpubs/FIPS/NIST.FIPS.180-4.pdf)


This function takes:
- the **current 256-bit hash value**, and  
- the **next 512-bit message block**,  
and produces the **updated hash value**.  

It is the core step that SHA-256 repeats for every message block.

---

### What the Compression Function Does 

1. **Build the Block (W[0..63])**  
   - Take the **first 16 words** straight from the 512-bit block.  
   - Then the algorithm **creates 48 more words** using the σ₀ and σ₁ functions.  
   - This gives us **64 words total**, because SHA-256 does 64 rounds.

2. **Set up the working variables**  
   - Copy the current hash values into:  
     `a, b, c, d, e, f, g, h`  
     These are just temporary variables used during the mixing.

3. **Do 64 mixing rounds**  
   - Each round uses the W words, constants K[t], and the functions Σ₀ (Sigma0), Σ₁ (Sigma1), Ch, Maj (From Problem 1)
   - The variables `a..h` change a little bit each round.

4. **Produce the new hash value**  
   - After the 64 rounds finish, the final `a..h` values are **added back** to the original hash.  
   - This becomes the **updated hash**, ready for the next block (if there is one).

---

### Simply Said


The compression function is the step that actually performs the hashing.  

It takes one padded 512-bit block of the message and the current hash value, expands the block into 64 words, runs 64 rounds of calculations, and then combines the result with the current hash. 

The output becomes the new hash value, which will be used for the next block or returned as the final SHA-256 hash if this is the only block.


---

### Example 

| Stage | Output (simplified) |
|-------|----------------------|
| Message | `"cat"` |
| After padding | 1 × 512-bit block `63617480 … 00000018` |
| W[0..15] | W[0] = 63617480<br>W[1] = 00000000<br>W[2] = 00000000<br>…<br>W[14] = 00000000<br>W[15] = 00000018 |
| Rounds 0–63 | State updated |
| Final hash | `d077f244de...` |

(That is the real SHA-256 of `"cat"`.)

---

### References Used

- FIPS 180-4 Secure Hash Standard :contentReference[oaicite:1]{index=1}  
- Functions from Problem 1 (Ch, Maj, Σ₀, Σ₁, σ₀, σ₁)  
- Constants from Problem 2  
- Padding & block parsing from Problem 3  
- [*ChatGPT (OpenAI)*](https://chatgpt.com/) – Interactive explanations, guidance, markdown used throughout this solution.


#### Step 1: Convert a 512-bit block into W[0..15]

According to the Secure Hash Standard (FIPS 180-4, §5.2.1 and §6.2.2), a 512-bit message block is treated as **sixteen 32-bit words** in big-endian order. This step takes a padded 512-bit block (64 bytes) and splits it into the first sixteen words of the message schedule:

- Each word is 4 bytes (32 bits).
- We read the block 4 bytes at a time.
- Each group of 4 bytes is converted to a 32-bit unsigned integer using big-endian byte order.
- The result is an array `W[0..15]` that we will later expand to `W[0..63]`.


In [ ]:
def initial_words_from_block(block: bytes) -> np.ndarray:
    """
    Convert a 512-bit (64-byte) message block into the first
    sixteen 32-bit words W[0..15] for SHA-256.

    Each word is read in big-endian order, as specified in
    FIPS 180-4 (see §5.2.1 and §6.2.2):
    https://nvlpubs.nist.gov/nistpubs/FIPS/NIST.FIPS.180-4.pdf

    Parameters
    ----------
    block : bytes
        A single 512-bit message block (exactly 64 bytes) produced
        after padding and parsing (See Problem 3).

    Returns
    -------
    numpy.ndarray
        A NumPy array of shape (16,) with dtype uint32, containing
        the words W[0]..W[15].
    """
    # Error Handling: SHA-256 compression always operates on 64-byte blocks.
    if len(block) != 64:
        raise ValueError(f"Block must be exactly 64 bytes (got {len(block)})")

    # Allocate an array of 16 unsigned 32-bit integers for W[0..15].
    W = np.zeros(16, dtype=np.uint32)

    # Process the block 4 bytes at a time (big-endian) to build W[0..15].
    for i in range(16):
        # Take 4 bytes: block[i*4 : (i+1)*4]
        word_bytes = block[i * 4 : (i + 1) * 4]

        # Convert those 4 bytes into a 32-bit integer (big-endian).
        W[i] = int.from_bytes(word_bytes, byteorder="big")

    return W


In [ ]:
# Example test for Step 1: using a known 512-bit block in hex.
# e.g. "63617480....00000018" (128 hex characters = 64 bytes).

blocks = list(block_parse(b"cat")) # Generate blocks for "cat" from Problem 3 function
block = blocks[0]   

print(f"Block length (bytes): {len(block)}")  # should be 64

# Convert block into W[0..15]
W_0_15 = initial_words_from_block(block)

print("\nW[0..15] as hex:")
for i, w in enumerate(W_0_15):
    print(f"W[{i:2}] = {w:08x}")



Block length (bytes): 64

W[0..15] as hex:
W[ 0] = 63617480
W[ 1] = 00000000
W[ 2] = 00000000
W[ 3] = 00000000
W[ 4] = 00000000
W[ 5] = 00000000
W[ 6] = 00000000
W[ 7] = 00000000
W[ 8] = 00000000
W[ 9] = 00000000
W[10] = 00000000
W[11] = 00000000
W[12] = 00000000
W[13] = 00000000
W[14] = 00000000
W[15] = 00000018


## <strong>Problem 5 </strong>: Passwords